required credentials
- Client ID
- Client Secret
- Password
- user_agent
- username


In [9]:
import os
import praw
from dotenv import find_dotenv, load_dotenv

In [8]:
load_dotenv(find_dotenv())

CLIENT_ID=os.getenv("CLIENT_ID")
SECRET_TOKEN=os.getenv("SECRET_TOKEN")
USERNAME=os.getenv("USERNAME")
PASSWORD=os.getenv("PASSWORD")

In [ ]:
reddit = 